<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/infer_condition2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
This condition will not ask users what type of inference they want. It will generate inferences, and rate the uncommonness and sensitivity of the inferences.

In [ ]:
import openai

openai.api_key = 'read api key'

# tokenizer = AutoTokenizer.from_pretrained("gpt2")  # GPT-3.5 & GPT-4 use GPT-2 tokenizer

def count_tokens(text):
    """Returns the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def truncate_list(data_list, max_tokens):
    """Truncates a list to fit within a token budget."""
    truncated_list = []
    token_count = 0
    for item in data_list:
        item_tokens = count_tokens(str(item))  # Count tokens in each item
        if token_count + item_tokens <= max_tokens:
            truncated_list.append(item)
            token_count += item_tokens
        else:
            break
    return truncated_list

def run_prompt(prompt):
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",  # Changed 'engine' to 'model'
        model="gpt-4o-mini",
        # model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        # max_tokens=min(1000, 16385 - input_token_count)  # Ensure within token limit
        temperature=0.7,
        n=1, # Return only one response
        stop=None # optional stopping sequence
        # request_timeout = 60
        )
    json_response = response.choices[0].message.content.strip()

    return json_response  # Convert response to Python dictionary

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import json

Mounted at /content/drive


In [ ]:
def write_json(output_filename, inference_json):
    if not inference_json:
        print("⚠️ No data to write. JSON file not created.")
        return None

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_json, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON saved successfully as {output_filename}")
    return output_filename

In [ ]:
def save_inference_json(inference_data, inference_no, columns_used):
    """Saves the inference data to a JSON file with an automated filename."""
    base_dir = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data"

    # Create a descriptive filename based on inference number and columns
    column_names = "_".join([col.split("_")[1] for col in columns_used])
    filename = f"inferences_{inference_no}_{len(columns_used)}cols_{column_names}.json"
    output_filename = f"{base_dir}/{filename}"

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_data, f, indent=4, ensure_ascii=False)

    print(f"✅ JSON saved successfully as {output_filename}")

In [ ]:
powerset2023 = pd.read_csv('/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_by_year/allActivity_2023.csv')

In [ ]:
# Select columns with less than 600 non-NaN rows
misc_columns = powerset2023.columns[powerset2023.notna().sum() < 600]

# Combine values into a new column
powerset2023['takeout1_misc_MyActivity_Search Title'] = powerset2023[misc_columns].apply(
    lambda row: ', '.join(row.dropna().astype(str)),
    axis=1)

# Drop the identified misc_columns
powerset2023 = powerset2023.drop(columns=misc_columns)


# Drop duplicate values in the specified column, keeping the first occurrence
powerset2023 = powerset2023.drop_duplicates(subset=['takeout1_chrome_MyActivity_Search Title'], keep='first')

make prompt more dynamic --> ask for user input for inference and recommendation #

*prompt: recommend one product based on these three inferences*

make numeric instead of text --> parameterize the model --> multiply it out --> if you recommend one product based on the inferences; for example inference 1 recommendation 1, inference 1 recommendation 2 and so on

In [ ]:
#Ask to select any four column
print("\nPlease select four datasets from the options (datasets should not be the same):")
for i, col in enumerate(powerset2023.columns, 1):
    print(f"{i}. {col}")

# Ask the user to select any 4 columns
selected_columns = []
df_selected = pd.DataFrame()
while len(selected_columns) < 4:
    try:
        col_num = int(input(f"\nSelect column {len(selected_columns) + 1} by number (1-{len(powerset2023.columns)}): "))
        if 1 <= col_num <= len(powerset2023.columns):
            col_name = powerset2023.columns[col_num - 1]
            if col_name not in selected_columns:
                selected_columns.append(col_name)
            else:
                print("You've already selected this column. Try another.")
        else:
            print("Invalid number. Please choose a number from the list.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the selected columns
powerSubset2023 = powerset2023[selected_columns]

# Print the selected DataFrame
print("\nSelected DataFrame:")


Please select four datasets from the options (datasets should not be the same):
1. takeout1_imageSearch_MyActivity_Search Title
2. takeout1_chrome_MyActivity_Search Title
3. takeout1_maps_MyActivity_Search Title
4. takeout1_YT_search-history_Search Title
5. takeout1_YT_watch-history_Search Title
6. takeout1_misc_MyActivity_Search Title

Select column 1 by number (1-6): 1

Select column 2 by number (1-6): 2

Select column 3 by number (1-6): 4

Select column 4 by number (1-6): 6

Selected DataFrame:


In [ ]:
# # Optionally, save to a CSV file
save_option = input("\nWould you like to save this selection to a CSV file? (yes/no): ").strip().lower()
if save_option == "yes":
    df_selected.to_csv("/content/drive/MyDrive/Dissertation/Study 1/Inference_data/24APRIL2025/chrome_maps_YTwatch_misc.csv", index=False)
    # print("File saved as 'selected_input_data.csv'.")


Would you like to save this selection to a CSV file? (yes/no): no


In [ ]:
#Ask to select how many inferences they wat

# Define the options
options = [3,6,9,12]

# Display the options
print("Please select a number from the options below by typing the corresponding number:\n")
for i, option in enumerate(options, 1):
    print(f"{i}. {option}")

# Get user input
user_input = input("\nEnter your selection (1-4): ")

# Validate input
if user_input.isdigit() and 1 <= int(user_input) <= 4:
    inference_no = options[int(user_input) - 1]
    print(f"\nYou selected: {inference_no}")
else:
    print("\nInvalid selection. Please enter a number between 1 and 4.")

# inference_input = int(inference_input)

Please select a number from the options below by typing the corresponding number:

1. 3
2. 6
3. 9
4. 12

Enter your selection (1-4): 1

You selected: 3


In [ ]:
# Initialize list to store responses
inference_c0_json = []

# Iterate through each column in power_subset1_
for col_name, col_data in powerSubset2023[:7500].items():
  # print(col_data.dropna().tolist())
  # Define the strict JSON prompt

  prompt = f"""User wants to know their interests based on {col_data.dropna().tolist()} in {col_name}
  1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
  2. For each interest, rate uncommonness of the interest (scale 1-10, where 1 = most uncommon and 10 = most common).
  Uncommon refers to how distinctive digital interaction patterns. demographic characteristics

  3. For each interest, rate sensitivity of interest (scale 1–10, where 1 = least sensitive and 10 = most sensitive).
  Sensitivity refers to how likely an interest is to reveal personal, private, or stigmatized information about the user —
  especially information they may not want to be shared, inferred, or acted upon without their consent.
  It includes how much harm, discomfort, or unwanted exposure could result from the disclosure of that interest

  4. Recommend **EXACTLY ONE** product for *EACH* inferences from step 1
  5. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_name}**.
   **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   {{
    "columns": {col_name},
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "commonness": f"<Inference 1 commonness score: {i+1}>",
      "explanation_commonness": f"<Explanation for commonness score: {i+1}>",
      "sensitivity": f"<Inference 1 sensitivity score: {i+1}>",
      "explanation_sensitivity": f"<Explanation for sensitivity score: {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
   }}
  }}"""

  print(f"🔹 Sending prompt to GPT for column: {col_name}...")
  assistant_reply = run_prompt(prompt)
  # print(assistant_reply)
  inference_c0_json.append(assistant_reply)

🔹 Sending prompt to GPT for column: takeout1_chrome_MyActivity_Search Title...
🔹 Sending prompt to GPT for column: takeout1_maps_MyActivity_Search Title...
🔹 Sending prompt to GPT for column: takeout1_YT_watch-history_Search Title...
🔹 Sending prompt to GPT for column: takeout1_misc_MyActivity_Search Title...


In [ ]:
inference_c0_json_copy = inference_c0_json.copy()

In [ ]:
def clean_json_string(json_string):
  """Cleans a JSON string by removing unwanted characters and standardizing spacing."""

  # cleaned = re.sub(r"```json ", "", json_string)
  # Replace extra spaces and newlines with single spaces
  cleaned = re.sub(r"```json|```", " ", json_string)
  # Remove characters outside the basic multilingual plane
  cleaned = re.sub(r"[^\u0000-\uFFFF]", "", cleaned)
  # Try to parse the cleaned JSON; return original if invalid
  try:
    return json.loads(cleaned)
  except json.JSONDecodeError:
    print(f"Warning: Failed to parse JSON, returning original: {json_string[:50]}...")
    return cleaned

# Apply the function to each item in your list

In [ ]:
cleaned_reply_c0 = [clean_json_string(item) for item in inference_c0_json]

In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c0_i3.json"
write_json(output_filename, cleaned_reply_c0)

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c0_i3.json


'/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c0_i3.json'

In [ ]:
inference_c2 = []
inference_c2_json = []

index = 0  # Keeps col_name[index] constant
col_names = list(powerSubset2023[:7500].columns)  # Convert column names to a list

while index < len(col_names):  # Outer loop for the constant column
    for next_index in range(index + 1, len(col_names)):  # Inner loop for incrementing column
        # Fetch data for the columns
        col_data_1 = powerSubset2023[:4500][col_names[index]]
        col_data_2 = powerSubset2023[:4500][col_names[next_index]]

        # Define the prompt
        prompt = f"""User wants to know what {col_data_1.dropna().tolist()} in {col_names[index]} and {col_data_2.dropna().tolist()} in {col_names[next_index]} tells about them.
        1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
        2. For each interest, rate uncommonness of the interest (scale 1-10, where 1 = most uncommon and 10 = most common). Uncommon refers to how rare or atypical an interest is among people who share similar demographic characteristics
        3. For each interest, rate sensitivity of interest (scale 1–10, where 1 = least sensitive and 10 = most sensitive). Sensitivity refers to how likely an interest is to reveal personal, private, or stigmatized information about the user — especially information they may not want to be shared, inferred, or acted upon without their consent. It includes how much harm, discomfort, or unwanted exposure could result from the disclosure of that interest
        4. Recommend **EXACTLY ONE** product for *EACH GROUP* identified in step 3
        5. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** in **{col_names[index]} AND {col_names[next_index]}.**

   **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": "{col_names[index]} AND {col_names[next_index]}",
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "commonness": f"<Inference 1 commonness score: >",
      "explanation_commonness": f"<Explanation for commonness score: >",
      "sensitivity": f"<Inference 1 sensitivity score: {i+1}>",
      "explanation_sensitivity": f"<Explanation for sensitivity score: {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
    }}
   }}
   ```
  }}

    **Strictly return a valid JSON object in this format. DO NOT include any other text or explanations.**"""

        # Run the prompt and get the response
        print(f"🔹 Sending prompt to GPT for column: **{col_names[index]} AND {col_names[next_index]}.**...")
        assistant_reply = run_prompt(prompt)
        # print(assistant_reply)
        inference_c2_json.append(assistant_reply)

        # if assistant_reply:
        #     # Remove unwanted code block markers
        #     cleaned_json = re.sub(r"```json\s*|\s*```", "", assistant_reply).strip()

        #     try:
        #         # Parse JSON
        #         parsed_json = json.loads(cleaned_json)
        #         inference_c2_json.append(parsed_json)  # Store parsed data
        #     except json.JSONDecodeError:
        #         print(f"⚠️ JSON Decoding Error. Skipping entry:\n{cleaned_json[:100]}...")  # Preview problematic JSON


    # Move to the next constant column
    index += 1

🔹 Sending prompt to GPT for column: **takeout1_chrome_MyActivity_Search Title AND takeout1_maps_MyActivity_Search Title.**...
🔹 Sending prompt to GPT for column: **takeout1_chrome_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title.**...
🔹 Sending prompt to GPT for column: **takeout1_chrome_MyActivity_Search Title AND takeout1_misc_MyActivity_Search Title.**...
🔹 Sending prompt to GPT for column: **takeout1_maps_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title.**...
🔹 Sending prompt to GPT for column: **takeout1_maps_MyActivity_Search Title AND takeout1_misc_MyActivity_Search Title.**...
🔹 Sending prompt to GPT for column: **takeout1_YT_watch-history_Search Title AND takeout1_misc_MyActivity_Search Title.**...


In [ ]:
cleaned_reply_c2 = [clean_json_string(item) for item in inference_c2_json]

In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c2_i3.json"
write_json(output_filename, cleaned_reply_c2)

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c2_i3.json


'/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c2_i3.json'

In [ ]:
inference_c3 = []
inference_c3_json = []

index = 0  # Keeps col_name[index] constant
col_names = list(powerSubset2023[:7500].columns)  # Convert column names to a list

# Outer loop keeps col_name[index] constant
while index < len(col_names):
    for next_index in range(index + 1, len(col_names) - 1):  # Iterate for combinations
        # Fetch data for the selected columns
        col_data_1 = powerSubset2023[:2500][col_names[index]]
        col_data_2 = powerSubset2023[:2500][col_names[next_index]]
        col_data_3 = powerSubset2023[:2500][col_names[next_index + 1]]

        # Define the prompt
        prompt = f"""User wants to know what {col_data_1.dropna().tolist()} in {col_names[index]} AND {col_data_2.dropna().tolist()} in {col_names[next_index]} AND {col_data_3.dropna().tolist()} in {col_names[next_index+1]} tells about them.
        1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
        2. For each interest, rate uncommonness of the interest (scale 1-10, where 1 = most uncommon and 10 = most common). Uncommon refers to how rare or atypical an interest is among people who share similar demographic characteristics
        3. For each interest, rate sensitivity of interest (scale 1–10, where 1 = least sensitive and 10 = most sensitive). Sensitivity refers to how likely an interest is to reveal personal, private, or stigmatized information about the user — especially information they may not want to be shared, inferred, or acted upon without their consent. It includes how much harm, discomfort, or unwanted exposure could result from the disclosure of that interest\
        4. Recommend **EXACTLY ONE** product for *EACH GROUP* identified in step 2
        5. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}**.

   **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": {col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]},
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "commonness": f"<Inference 1 commonness score: >",
      "explanation_commonness": f"<Explanation for commonness score: >",
      "sensitivity": f"<Inference 1 sensitivity score: {i+1}>",
      "explanation_sensitivity": f"<Explanation for sensitivity score: {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
    }}
   }}
   ```
  }}

    **Strictly return a valid JSON object in this format. DO NOT include any other text or explanations.**"""

        print(f"🔹 Sending prompt to GPT for column: {col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}...")
        assistant_reply = run_prompt(prompt)
        # print(assistant_reply)

        if assistant_reply:
            # Remove unwanted code block markers
            cleaned_json = re.sub(r"```json\s*|\s*```", "", assistant_reply).strip()

            try:
                # Parse JSON
                parsed_json = json.loads(cleaned_json)
                inference_c3_json.append(parsed_json)  # Store parsed data
            except json.JSONDecodeError:
                print(f"⚠️ JSON Decoding Error. Skipping entry:\n{cleaned_json[:100]}...")  # Preview problematic JSON

    index += 1

🔹 Sending prompt to GPT for column: takeout1_chrome_MyActivity_Search Title AND takeout1_maps_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title...
🔹 Sending prompt to GPT for column: takeout1_chrome_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title AND takeout1_misc_MyActivity_Search Title...
🔹 Sending prompt to GPT for column: takeout1_maps_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title AND takeout1_misc_MyActivity_Search Title...


In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c3_i3.json"
write_json(output_filename, inference_c3_json)

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c6_i3.json


'/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c6_i3.json'

In [ ]:
inference_c3_json

In [ ]:
inference_c4 = []
inference_c4_json = []

index = 0  # Keeps col_name[index] constant
col_names = list(powerSubset2023[:7500].columns)  # Convert column names to a list

# Outer loop keeps col_name[index] constant
while index < len(col_names):
    for next_index in range(index + 1, len(col_names) - 2):  # Iterate for combinations
        # Fetch data for the selected columns
        col_data_1 = powerSubset2023[:2500][col_names[index]]
        col_data_2 = powerSubset2023[:2500][col_names[next_index]]
        col_data_3 = powerSubset2023[:2500][col_names[next_index + 1]]
        col_data_4 = powerSubset2023[:2500][col_names[next_index + 2]]

        # Define the prompt
        prompt = f"""User wants to know what {col_data_1.dropna().tolist()} in {col_names[index]} AND {col_data_2.dropna().tolist()} in {col_names[next_index]} AND {col_data_3.dropna().tolist()} in {col_names[next_index+1]} AND {col_data_4.dropna().tolist()} in {col_names[next_index+2]} tells about them.
        1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
        2. For each interest, rate uncommonness of the interest (scale 1-10, where 1 = most uncommon and 10 = most common). Uncommon refers to how rare or atypical an interest is among people who share similar demographic characteristics
        3. For each interest, rate sensitivity of interest (scale 1–10, where 1 = least sensitive and 10 = most sensitive). Sensitivity refers to how likely an interest is to reveal personal, private, or stigmatized information about the user — especially information they may not want to be shared, inferred, or acted upon without their consent. It includes how much harm, discomfort, or unwanted exposure could result from the disclosure of that interest\
        4. Recommend **EXACTLY ONE** product for *EACH GROUP* identified in step 2
        5. Recommend **EXACTLY ONE** product based on the **combination of all {inference_no} inferences** for **{col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}**.

   **STRICT JSON OUTPUT FORMAT (No extra text or explanations, just JSON):**
   ```json
   {{
    "columns": {col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]} AND {col_names[next_index+2]},
    "inferences": [
    {', '.join([json.dumps({
      "inference": f"<output for inference {i+1}>",
      "explanation_inference": f"<explanation for inference {i+1}>",
      "commonness": f"<Inference 1 commonness score: >",
      "explanation_commonness": f"<Explanation for commonness score: >",
      "sensitivity": f"<Inference 1 sensitivity score: {i+1}>",
      "explanation_sensitivity": f"<Explanation for sensitivity score: {i+1}>",
      "recommendation": f"<recommendation for inference {i+1}>"
    }) for i in range(inference_no)])}
    ],
    "final_product_recommendation": {{
      "ONE recommendation based on ALL inferenceces": "<Product name and company>",
    }}
   }}
   ```
  }}

    **Strictly return a valid JSON object in this format. DO NOT include any other text or explanations.**"""

        print(f"🔹 Sending prompt to GPT for column: {col_names[index]} AND {col_names[next_index]} AND {col_names[next_index+1]}...")
        assistant_reply = run_prompt(prompt)
        print(assistant_reply)
        inference_c4_json.append(assistant_reply)

    index += 1

In [ ]:
cleaned_reply_c4 = [clean_json_string(item) for item in inference_c4_json]

In [ ]:
cleaned_reply_c4

[{'columns': 'takeout1_chrome_MyActivity_Search Title AND takeout1_maps_MyActivity_Search Title AND takeout1_YT_watch-history_Search Title AND takeout1_misc_MyActivity_Search Title',
  'inferences': [{'inference': 'interested in digital privacy and data protection',
    'explanation_inference': 'The user frequently searches for topics related to data privacy, personalized ads, and research on children’s privacy.',
    'commonness': '6',
    'explanation_commonness': 'Interest in digital privacy is relatively common, especially among individuals engaged in technology or education sectors.',
    'sensitivity': '8',
    'explanation_sensitivity': 'This interest could reveal personal beliefs about privacy and security which may be sensitive in discussions.',
    'recommendation': 'Privacy Tools | Harvard University Privacy Tools Project'},
   {'inference': 'interested in travel and local attractions',
    'explanation_inference': 'The user has viewed various locations in South Carolina, in

In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c4_i3.json"
write_json(output_filename, cleaned_reply_c4)

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c4_i3.json


'/content/drive/MyDrive/Dissertation/Study 1/Inference_data/01MAY2025/chrome_map_ytWatch_misc_data_c4_i3.json'

In [ ]:
index = 0

unCommon_Inferences = []

for col_name, col_data in power_subset1_.items():
  if index <= len(power_subset1_.columns):
    prompt = f"""User wants the model to analyze {col_data} in {col_name[index]}. Based on this data:
    1. Provide three unique inferences about this user based on the user’s distinctive digital interaction patterns. Make sure that these inferences reflect predicted characteristics of the user that are HIGHLY UNCOMMON in society.
    2. Recommend one product or solution for each inference.
    3. Explain briefly how you arrived at each inference
    4. Explain briefly how you arrived at each recommendation
    """
    prompt += presentData
    # print(prompt)

    assistant_reply = run_prompt(prompt)
    print(assistant_reply)

    for row in assistant_reply.split('\n'):
      if "|" in row:  # Only process rows with data
        elements = [cell.strip() for cell in row.split("|")]
        unCommon_Inferences.append(elements)
      # if len(elements) > 1:
      #           # Adjust the number of elements to match the expected columns
      #           elements = elements[:8]  # or the actual number of columns you expect

  index = index+1

Ask LLM while generating or retrospectively to rate each of these inferences as a level of uncommonness. Add a column with inference score of 1 to 10 for uncommonness. Higher score to uncommon inferences than some of the others.

Pick top N of the uncommon (unexpected), sensitive ones (including combinations) and use them

Robustness in repitition and generalizabiliy

In [ ]:
index = 0

sensitiveInferences_notFiltered = []

for col_name, col_data in power_subset1_.items():
  if index <= len(power_subset1_.columns):
    prompt = f"""user wants to know what {col_data} in {col_name[index]} tells about the following
    1. Three personal preference or characteristics which the user did not reveal, but analysis of their online activity revealed.
    2. How algorithm inferring personal preferences or characteristics can positively impact user
    3. Explain briefly how you made the inferences
    4. Recommend ONE relevant product OR service OR event OR activity that could reveal user's personal preferences or characteristics
    5. Explain how the inferences and recommendation violate the user's personal and data privacy
    """
    # print(prompt)

    prompt += f"""Present the inferences and recommendations in a table that looks like below:
    STRICTLY FOLLOW THE FORMAT BELOW:

    Data Point|Inferring Personal Preference or Characteristics | Explanation for Inference| Product Recommendation |Impact on users' personal and data privacy|   |
    {col_name}|Inference 1: <output for interference> | Inference 1: <explanation> | Recommendation 1: <recommendation> |privacy impact 1: <explain how it impacts privacy>|
    {col_name}|Inference 2: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 2: <explain how it impacts privacy>|
    {col_name}|Inference 3: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 3: <explain how it impacts privacy>|
    """
    # prompt = f"""Analyze the user's online activity data and provide inferences based on the column '{col_name[index]}' and its values '{col_data}'.

    # Your task:
    # 1. Identify **three personal preferences or characteristics** that the user has not explicitly stated but can be inferred from the data.
    # 2. Explain how **algorithmic inference** of such data can be beneficial to the user.
    # 3. Briefly **describe the reasoning** behind each inference.
    # 4. Suggest **one relevant product, service, event, or activity** that aligns with the inferred preferences.
    # 5. Explain how these inferences and recommendations **pose risks to user privacy or could be considered harmful**.

    # **Present the results in a structured table** following this exact format:

    # STRICTLY FOLLOW THE FORMAT BELOW:

    # Data Point|Inferring Personal Preference or Characteristics | Explanation for Inference| Product Recommendation |Impact on users' personal and data privacy|   |
    # {col_name}|Inference 1: <output for interference> | Inference 1: <explanation> | Recommendation 1: <recommendation> |privacy impact 1: <explain how it impacts privacy>|
    # {col_name}|Inference 2: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 2: <explain how it impacts privacy>|
    # {col_name}|Inference 3: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 3: <explain how it impacts privacy>|

    # Ensure that all inferences, explanations, and recommendations are logically derived from the given data. Be concise yet informative."""

    assistant_reply = run_prompt(prompt)
    # print(assistant_reply)

    for row in assistant_reply.split('\n'):
      if "|" in row:  # Only process rows with data
        elements = [cell.strip() for cell in row.split("|")]
        sensitiveInferences_notFiltered.append(elements)
      # if len(elements) > 1:
      #           # Adjust the number of elements to match the expected columns
      #           elements = elements[:8]  # or the actual number of columns you expect

  index = index+1

In [ ]:
sensitiveInferences_notFiltered_df = pd.DataFrame(sensitiveInferences_notFiltered)

In [ ]:
sensitiveInferences_notFiltered_df

## Filter Data and then Make Sensitive Inference

In [ ]:
# import openai
from transformers import AutoTokenizer

inference_c2 = []
index = 0
col_names = list(power_subset1_.columns)

while index < len(col_names):
    for next_index in range(index + 1, len(col_names)):
        col_data_1 = power_subset1_[col_names[index]].dropna().tolist()
        col_data_2 = power_subset1_[col_names[next_index]].dropna().tolist()

        # Static part of the prompt (excluding data)
        base_prompt = f"""user wants to know what [DATA] in {col_names[index]} and [DATA] in {col_names[next_index]} tells about them as per the following:
        1. Three inferences about the user based on their online behavior in the form interested in X (where X is a single interest) for each column.
        2. Recommend a product for each inference in each column.
        3. Recommend ONE product based of the three inference for example, based on inferences 1, 2, and 3, i recommend the product X
        4. Explain briefly how you arrived at each inference for each column.
        5. Explain briefly how you arrived at each recommendation for each inference.
        6. Save all inferences and recommendation in ONE table

        STRICTLY FOLLOW THE FORMAT BELOW:

        {col_names[index]} and {col_names[next_index]}|Inference 1: <output for inference>| Recommendation for inference 1| Recommendation based on inference 1, 2, and 3| Explanation for inference 1| Explanation for recommendation 1|
        {col_names[index]} and {col_names[next_index]}|Inference 2: <output for inference>| Recommendation for inference 2| Recommendation based on inference 1, 2, and 3| Explanation for inference 2| Explanation for recommendation 2|
        {col_names[index]} and {col_names[next_index]}|Inference 3: <output for inference>| Recommendation for inference 3| Recommendation based on inference 1, 2, and 3| Explanation for inference 3| Explanation for recommendation 3|
        """

        # Count tokens in base prompt
        base_prompt_tokens = count_tokens(base_prompt)

        # Ensure we don't exceed model limit (16,385 tokens)
        max_allowed_tokens = 16_385 - base_prompt_tokens

        # Split available tokens between col_data_1 and col_data_2
        max_tokens_per_list = max_allowed_tokens // 2

        # Truncate data to fit within token limit
        col_data_1_truncated = truncate_list(col_data_1, max_tokens_per_list)
        col_data_2_truncated = truncate_list(col_data_2, max_tokens_per_list)

        # Construct final prompt
        prompt = f"""user wants to know what {col_data_1_truncated} in {col_names[index]} and {col_data_2_truncated} in {col_names[next_index]} tells about them as per the following:
        1. Three inferences about the user based on their online behavior in the form interested in X (where X is a single interest) for each column.
        2. Recommend a product for each inference in each column.
        3. Recommend ONE product based of the three inference for example, based on inferences 1, 2, and 3, i recommend the product X
        4. Explain briefly how you arrived at each inference for each column.
        5. Explain briefly how you arrived at each recommendation for each inference.
        6. Save all inferences and recommendation in ONE table

        STRICTLY FOLLOW THE FORMAT BELOW:

        {col_names[index]} and {col_names[next_index]}|Inference 1: <output for inference>| Recommendation for inference 1| Recommendation based on inference 1, 2, and 3| Explanation for inference 1| Explanation for recommendation 1|
        {col_names[index]} and {col_names[next_index]}|Inference 2: <output for inference>| Recommendation for inference 2| Recommendation based on inference 1, 2, and 3| Explanation for inference 2| Explanation for recommendation 2|
        {col_names[index]} and {col_names[next_index]}|Inference 3: <output for inference>| Recommendation for inference 3| Recommendation based on inference 1, 2, and 3| Explanation for inference 3| Explanation for recommendation 3|
        """

        # Run the prompt and get response
        assistant_reply = run_prompt(prompt)
        print(assistant_reply)

        for row in assistant_reply.split("\n"):
            elements = [e.strip() for e in row.strip("|").split("|") if e.strip()]
            if len(elements) >= 5 and elements[0].strip() and not all(e.strip().startswith("-") for e in elements):
                inference_c2.append([e.strip() for e in elements])

    index += 1  # Move to next constant column


KeyboardInterrupt: 

## Filter Sensitive Data

In [ ]:
# Initialize filter_sensitive_data list
filter_sensitive_data_ = []

# Iterate through the DataFrame
for col_name, col_data in power_subset1_.items():
    # Drop NaN values
    col_data_cleaned = col_data.dropna().reset_index(drop=True)

    # Generate prompt
    prompt = f"""
    The user wants to know which entries in `{col_name}`:
    1. When analyzed, can accurately identify private preferences, features, or characteristics of the user whose online activity is analyzed.
       For example, when a user is shopping for diamond jewelry, it could be used to infer the financial status of the user.
    2. EXCLUDE ALL `NaN` values.
    3. Provide a concise explanation of why `{col_name}` is considered sensitive.
    4. Make three inferences that can potentially reveal  user's private preferences, features, or characteristics
    5. Explain briefly how you arrived at each inference.
    Present the data and explanation in a table. STRICTLY FOLLOW THIS FORMAT:

    | Sensitive Data in `{col_name}` | Explanation for `{col_name}`                   |
    |-------------------------------|-------------------------------------------------|
    """

    # Add each entry to the prompt
    for entry in col_data_cleaned:
        prompt += f"| {entry} | Data point is sensitive because: <Your explanation> | \n"


    # Simulate API call with the prompt (replace with run_prompt in production)
    assistant_reply = run_prompt(prompt)  # Simulating the assistant's reply with the generated prompt for now.

    # Parse the reply
    for row in assistant_reply.split("\n"):
        if "|" in row:
            elements = [cell.strip() for cell in row.split("|") if cell.strip()]
            if len(elements) == 2:  # Only process valid rows
                filter_sensitive_data_.append({"Column": col_name, "Sensitive Data": elements[0], "Explanation": elements[1]})

In [ ]:
df_sd_ = pd.DataFrame(filter_sensitive_data_)

# news_data = df_sd_[df_sd_['Column'] == 'news_article'][['Sensitive Data', 'Explanation']].reset_index(drop=True)
youtube_data = df_sd_[df_sd_['Column'] == 'action_info'][['Sensitive Data', 'Explanation']].reset_index(drop=True)
chrome_data = df_sd_[df_sd_['Column'] == 'chromeAction_info'][['Sensitive Data', 'Explanation']].reset_index(drop=True)
map_data = df_sd_[df_sd_['Column'] == 'mapAction_info'][['Sensitive Data', 'Explanation']].reset_index(drop=True)
shop_data = df_sd_[df_sd_['Column'] == 'shop_activity'][['Sensitive Data', 'Explanation']].reset_index(drop=True)

# Merge all the data into one DataFrame
sensitiveData_df = pd.concat([youtube_data, chrome_data, map_data, shop_data], axis=1)

# Renaming the columns for clarity
sensitiveData_df.columns = ['Youtube Action Sensitive Data', 'Youtube Action Explanation',
                     'Chrome Action Sensitive Data', 'Chrome Action Explanation',
                     'Map Action Sensitive Data', 'Map Action Explanation',
                     'Shop Activity Sensitive Data', 'Shop Activity Explanation']

sensitiveData_df.drop(index=[0,1], inplace=True)

## Inferences from Sensitive Data

In [ ]:
# col_names = list(sensitiveData_df.columns)

index = 0
sensitive_inferences = sensitiveData_df[['Youtube Action Sensitive Data', 'Chrome Action Sensitive Data', 'Map Action Sensitive Data', 'Shop Activity Sensitive Data']]
# sensitive_inferences
sensitive_inferences_ = []

for col_name, col_data in sensitive_inferences.items():
  if index <= len(sensitive_inferences.columns):
    prompt = f"""user wants to know what {col_data} in {col_name[index]} tells about the following
    1. Three personal preference or characteristics which the user did not reveal, but analysis of their online activity revealed.
    2. How algorithm inferring personal preferences or characteristics can positively impact user
    3. Explain briefly how you made the inferences
    4. Recommend ONE relevant product OR service OR event OR activity that could reveal user's personal preferences or characteristics
    5. Explain how the inferences and recommendation violate the user's personal and data privacy
    """
    # print(prompt)

    prompt += f"""Present the inferences and recommendations in a table that looks like below:
    STRICTLY FOLLOW THE FORMAT BELOW:

    Data Point|Inferring Personal Preference or Characteristics | Explanation for Inference| Product Recommendation |Impact on users' personal and data privacy|   |
    {col_name}|Inference 1: <output for interference> | Inference 1: <explanation> | Recommendation 1: <recommendation> |privacy impact 1: <explain how it impacts privacy>|
    {col_name}|Inference 2: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 2: <explain how it impacts privacy>|
    {col_name}|Inference 3: <output for interference> | Inference 2: <explanation> | Recommendation 1: <recommendation> |privacy impact 3: <explain how it impacts privacy>|
    """

    assistant_reply = run_prompt(prompt)
    # print(assistant_reply)
    sensitive_inferences_.append(assistant_reply)

  index = index+1

In [ ]:
all_data = []
for reply in sensitive_inferences:
    for row in reply.split('\n'):
        if "|" in row:  # Only process rows with data
            elements = [cell.strip() for cell in row.split("|")]
            if len(elements) > 1:
                # Adjust the number of elements to match the expected columns
                elements = elements[:8]  # or the actual number of columns you expect
                all_data.append(elements)

# Create DataFrame
sensitive_inferences_df = pd.DataFrame(all_data)

# Dynamically determine the number of columns
num_cols = len(sensitive_inferences_df.columns)

# Set column names based on the actual number of columns
if num_cols == 7: # Change to 8 to match your expected DataFrame
    sensitive_inferences_df.columns = [
        "Data Point",
        "Inferring Personal Preference or Characteristics",
        "Explanation for Inference",
        "Product Recommendation",
        "Impact on users' personal and data privacy",
        "Extra Column 1",  # Add names for extra columns
        "Extra Column 2"   # Add names for extra columns
    ]
# You might want to add more else-if conditions for different numbers of columns
# Or use a more generic approach to handle extra columns.


In [ ]:
sensitive_inferences_df

In [ ]:
sensitive_inferences_df.to_csv('/content/drive/MyDrive/Dissertation/Study 1/Inference_data/sensitive_inferences_iter7.csv', index=False)